In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from transformers import BertTokenizer
from sklearn.utils import shuffle

from typing import *

from MEOW_Models import Classification_models, MT_models
from MEOW_Utils import Classification_utils, MT_utils

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 4
EPOCH_NUM = 10
INPUT_FILE_PATH = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Sentiment_train.csv'
PRETRAINED_MODULE_NAME = 'bert-base-uncased'

In [7]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODULE_NAME)
train_df = MT_utils.get_Sentiment_df(INPUT_FILE_PATH)
num_labels = len(train_df.value_counts('label').keys())
train_df.sample(8)

,context,label,label_name
11397,IÂm getting personal... IÂm not one to allow...,4,Extremely Negative


In [16]:
train_df.head()

,context,label,label_name
0,What are my rights with event/travel cancellat...,1,Positive
1,"@FrischepostHH delivers food to front door, of...",2,Neutral
2,GREAT NEWS!\r\r\n\r\r\nREDUCED PRICES DUE TO #...,0,Extremely Positive
3,National #supermarket chains have failed the #...,4,Extremely Negative
4,How is your business adapting to #COVID19? Can...,0,Extremely Positive


In [5]:
bert_model = MT_models.BertWithoutEmbedding.from_pretrained(PRETRAINED_MODULE_NAME)
GETEMBEDDING_helper = MT_utils.get_bert_element(bertmodel=bert_model)
embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

train_dataset = Classification_utils.Classification_dataset(df = train_df, tokenizer = tokenizer, num_labels=num_labels)
train_loader = Classification_utils.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=Classification_utils.collate_batch)

Classification_model = Classification_models.Bert_classification(model=bert_model, embedding_layer=embedding_layer, device=DEVICE, num_labels=num_labels)
Classification_model.to(DEVICE)

optimizer = torch.optim.SGD(Classification_model.parameters(), lr=0.0001, momentum=0.9)
# optimizer = torch.optim.Adam(QA_model.parameters(), lr=0.001, betas=(0.88, 0.95), eps=1e-08)

H = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithoutEmbedding: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertWithoutEmbedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithoutEmbedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
print(train_dataset[0])

(tensor([  101,  2002,  2003,  6951,  2075,  2061,  2515,  2008,  3084,  2009,
         3100,  1029,  2054,  2055, 13025,  3573, 26706,  1010, 11500,  1010,
         7435,  1998, 13882,  3095,  1029,  2358,  8516,  2923,  2035,  2058,
         2024,  2701,  1012,  2339,  2003,  6951,  2075,  3100,  1029,  2064,
         1045,  2330,  2039,  2007, 11875,  1998,  7308,  1029,  1001,  2994,
         8988,  8462,  1001, 21887, 23350,  1001,  2522, 17258,  1001, 26629,
         1001,  2606, 16770,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
        16731,  2890, 17922,  2549,  2100, 23644,   102]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
for epoch in range(EPOCH_NUM):
    print("the {:d} iter :".format(epoch+1))

    Classification_model.train()
    # train 
    training_loss = 0.0
    training_correct = 0.0

    for data in train_loader:
        input_ids, mask, token, label = data
        
        # input_ids = input_ids.type(torch.IntTensor)
        # mask = mask.type(torch.IntTensor)
        # labels = labels.type(torch.LongTensor)

        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        label = label.to(DEVICE)
    
        outputs = Classification_model(input_ids, token=token, attention_mask=mask, label=label )
        loss = outputs[0]
        prob = outputs[1]

        print(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predict = torch.argmax(prob, dim=1)
        label = torch.argmax(label, dim=1)
        correct_num = (predict == label).type(torch.int).sum()
        
        training_loss += loss.item()
        training_correct += correct_num
        
        # print(correct_num)
        

    avg_loss = training_loss / len(train_loader)
    avg_acc = training_correct / len(train_dataset)

    H['train_loss'].append(avg_loss)
    H['train_acc'].append(avg_acc)
    print("Train loss: {:.6f}, Train accuracy {:.4f}".format(avg_loss, avg_acc))


the 1 iter :


IndexError: list assignment index out of range

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# tensor to float
H['train_loss'] = [float(i) for i in H['train_loss']]
# H['train_acc'] = [float(i) for i in H['train_acc']]

# H['val_loss'] = [float(i) for i in H['val_loss']]
# H['val_acc'] = [float(i) for i in H['val_acc']]

# loss
plt.figure()
plt.title("Loss")
plt.xlabel("EPOCH")
plt.ylabel("Loss")
plt.plot(H["train_loss"], label="test_loss")
# plt.plot(H["val_loss"], label="test_loss")
plt.xticks(np.arange(10), range(1,11,1))
plt.show()

# accuracy
# plt.figure()
# plt.title("Test Accuracy")
# plt.xlabel("EPOCH")
# plt.ylabel("Accuracy")
# plt.plot(H["train_acc"], label="test_acc")
# # plt.plot(H["val_acc"], label="test_acc")
# plt.xticks(np.arange(6), range(1,7,1))
# plt.show()

